# The Basic Concept of the Filtering

# <img src="https://i.imgur.com/zZ3wkbu.png" width=500  />

### 本章節內容大綱
* [Nosie 雜訊](#Nosie-雜訊)
  - [Gaussian Noise 高斯雜訊](#Gaussian-Noise-高斯雜訊)
  - [Impulse Noise (Salt-and-Pepper Noise) 椒鹽雜訊](#Impulse-Noise-(Salt-and-Pepper-Noise)-椒鹽雜訊)
* [Filter 濾波器](#Filter-濾波器)
  - [濾波器種類](#濾波器種類)
  - [平滑濾波器](#平滑濾波器)
    - [低通濾波器 Low Pass Filter](#平滑濾波器--低通濾波器-Low-Pass-Filter)
    - [中值濾波器 Median Filter](#平滑濾波器--中值濾波器-Median-Filter)
  - [高通濾波器](#高通濾波器-High-Pass-Filter)
    - [Sobel Filter -- Edge Detection](#Sobel-Filter----Edge-Detection)
    - [Laplacian Filter](#Laplacian-Filter)
    - [Laplacian -Sharpening 影像銳化](#Laplacian---Sharpening-影像銳化)

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import skimage
from numpy import random
from numpy import shape

## Nosie 雜訊 <img src="https://i.imgur.com/K86Itha.png" width=200  />

 - 在電子學中，訊號在傳輸過程中容易受到一些外在能量所產生訊號（如雜散電磁場等）的干擾，這些能量即雜訊。
 - 雜訊通常會造成訊號的失真。其來源除了來自系統外部，亦有可能由接收系統內部產生。
 - 雜訊在生活中的例子如，在收音機或是無線對講機發出「唦唦」的嘈雜聲，或者像是傳統電視顯示黑白色閃爍如「雪花」的畫面。
 - 影像是訊號的一種，因此在傳輸的過程中不免會產生雜訊，在整個影像學中，「雜訊的產生」與「去除雜訊」是整個影像學的柱石。

<img src="https://i.imgur.com/z7HfIiJ.png" width=250  />

我們簡單列舉幾個在教科書上常見的雜訊....<br>
<img src="https://i.imgur.com/0ePogoK.png" width=700  />
<img src="https://i.imgur.com/uNBxLjn.png" width=700  />

其中我們挑出最為常見的 Gaussian Noise 和 Impulse Noise 來說明<br>
<img src="https://i.imgur.com/Md6vIx1.png" width=700  />

 - ### Gaussian Noise 高斯雜訊
 Gaussian Noise 即是將雜訊轉換到 frequency domain 來看，它分佈的狀態是會呈現所謂的高斯分佈( Gaussian distribution )，如下圖：
 <img src="https://i.imgur.com/ro69DlW.png" width=500  />
 
 任何電子只要溫度在絕對零度 K 以上，就會產生振動，振動的電子就會產生隨機訊號，即稱作熱雜訊 Thermal Noise。於是科學家發現，熱雜訊在 frequency domain 的分佈恰好是呈現高斯分佈的，也就是 Gaussian Noise。常見的生活例子，如未調好的電視螢幕。

In [ ]:
# upload Einstein.jpg
!gdown --id 1xYyw5whxN2lgQPIOSAdO0GaW5M-yOXtK

In [ ]:
img = cv2.imread('Einstein.jpg')[:, :, ::-1]


def gasuss_noise(image, mean=0, var=0.004):
  image = np.array(image/255, dtype=float)
  noise = np.random.normal(mean, var ** 0.5, image.shape)
  out = image + noise
  out = np.clip(out, 0, 1.0)
  out = np.uint8(out * 255)
  return out


output = [img, gasuss_noise(img)]
titles = ['Original', 'Gaussian Noise']
plt.figure(figsize=(12, 6))
for i in range(2):
  plt.subplot(1, 2, i+1)
  plt.imshow(output[i])
  plt.title(titles[i], fontsize=16)
  plt.axis("off")
plt.show()

 - ### Impulse Noise (Salt-and-Pepper Noise) 椒鹽雜訊
 
如果訊號在成像或是傳輸過程中，突然被一個脈波 pulse 干擾到，就有機會產生 Impulse noise。而在影像上這種雜訊通常會以極值 0 或 255 出現，即一個小黑點 0 或小白點 255，看起來就像在影像上灑上了胡椒(黑)與鹽(白)，因此又稱 Salt-and-Pepper Noise。

In [ ]:
image = cv2.imread('Einstein.jpg')[:, :, ::-1]


def ImpulseNoise(img, noise_type, percetage):
  NoiseNum = int(percetage * img.shape[0] * img.shape[1])

  randX = np.random.randint(0, img.shape[0], NoiseNum)
  randY = np.random.randint(0, img.shape[1], NoiseNum)

  if noise_type == "pepper":
    img[randX, randY] = 0
  elif noise_type == "salt":
    img[randX, randY] = 255
  elif noise_type == "s&p":
    img[randX[::2], randY[::2]] = 0
    img[randX[1::2], randY[1::2]] = 255
  else:
    pass

  return img


noise_list = ["original", "salt", "pepper", "s&p"]

plt.figure(figsize=(18, 10))
for i, each in enumerate(noise_list):
  plt.subplot(1, 4, i+1)
  plt.imshow(ImpulseNoise(image.copy(), each, 0.1))
  plt.title(each, fontsize=15)
  plt.axis("off")
plt.show()

## Filter 濾波器

* Filter 邊長通常為奇數的方形，又稱為遮罩 Mask、Kernel。
* 主要概念是將遮罩覆蓋到指定影像的每個像素上面。<br>
<img src="https://drive.google.com/uc?export=view&id=1Ah8U3wiau3X4gMGFaaawIRpYBtjLebYk" width=550  /><br>

 * ## 濾波器種類
    - 平滑濾波器：用於模糊化和去除雜訊。
        - 低通濾波器 (Low pass filter)
        - 中值濾波器 (Median filter)
    -  銳化濾波器：強化物體的邊緣位置。
        - 高通濾波器 (High pass filter)
          - Sobel Filter
          - Laplacian Filter


* #### 平滑濾波器--低通濾波器 Low Pass Filter
 - 保持低頻部分，減少或消除高頻部分。
    - 高頻：短距離內灰階值變化值大(例：邊緣或雜訊)
    - 低頻：短距離內灰階值變化值小(例：背景或皮膚紋理)
 - 方法：將每一個像素的所有鄰近的灰階值加總起來，然後再以平均值取代該像素之灰階值。
 - 效果：將灰階變化較大之像素的值縮小，因此影像的「銳利」變化會降低，所以低通濾波器有模糊邊緣這個副作用。
 - 最常見的低通濾波器為 Average Filter、Weighted Average Filter
 
 <img src="https://i.imgur.com/7YAN07q.png" width=400  />

In [ ]:
img = cv2.imread('Einstein.jpg')[:, :, ::-1]

-- Average Filter 模糊化

In [ ]:
average_filter = np.ones((3, 3))/9
ave_img = cv2.filter2D(img, -1, average_filter)

In [ ]:
output = [img, ave_img]
titles = ['Original', 'Average Filter']
plt.figure(figsize=(16, 8))
for i in range(2):
  plt.subplot(1, 2, i+1)
  plt.imshow(output[i], cmap='gray')
  plt.title(titles[i], fontsize=16)
  plt.axis("off")
plt.show()

-- Average Filter 去雜訊

In [ ]:
img = cv2.imread('Einstein.jpg')[:, :, ::-1]/255.0

gau_img = skimage.util.random_noise(img.copy(),
                                    mode="gaussian",
                                    mean=0, var=0.001)
sp_img = skimage.util.random_noise(img.copy(), mode="s&p")

In [ ]:
average_filter = np.ones((3, 3))/9
ave_img = cv2.filter2D((gau_img.copy()*255).astype("uint8"),
                       -1, average_filter)

In [ ]:
output = [img, gau_img, ave_img]
titles = ['Original', 'Gaussian Noise', 'Average Filter']
plt.figure(figsize=(24, 8))
for i in range(3):
    plt.subplot(1, 3, i+1)
    plt.imshow(output[i], cmap='gray')
    plt.title(titles[i], fontsize=16)
    plt.axis("off")
plt.show()

* #### 平滑濾波器--中值濾波器 Median Filter
 - 效果：中值濾波器可以將較強的高頻雜訊去除，而仍然能夠保持邊緣的銳度。
 - 方法：將遮罩內之像素值進行排序，找出中間值，然後將遮罩內中像素之灰階值以該中間值取代。
 
 <img src="https://i.imgur.com/fUZsXWa.png" width=550  />

In [ ]:
median = cv2.medianBlur((sp_img.copy()*255).astype("uint8"), 3)

In [ ]:
output = [sp_img, median]
titles = ['Impulse Noise', 'Median Filter']
plt.figure(figsize=(16, 8))
for i in range(2):
  plt.subplot(1, 2, i+1)
  plt.imshow(output[i], cmap='gray')
  plt.title(titles[i], fontsize=16)
  plt.axis("off")
plt.show()

   - 低通濾波器具有去雜訊的效果，但同時在影像上也可能造成不同效果的模糊化，如下...

In [ ]:
image = cv2.imread('Einstein.jpg')[:, :, ::-1]

img_noise = ImpulseNoise(image.copy(), each, 0.1)

avg_blur = cv2.blur(image, (5, 5))
Gauss_blurred = cv2.GaussianBlur(image, (15, 15), 5)
median_blurred = cv2.medianBlur(image, 5)
bilateral_blur = cv2.bilateralFilter(image, 9, 75, 75)


titles = ['Original_img', 'PepperandSalt_img', 'avg_blur',
      'Gauss_blurred', 'median_blurred', 'bilateral_blur']
images = [image, img_noise, avg_blur, Gauss_blurred,
      median_blurred, bilateral_blur]
plt.figure(figsize=(18, 12))
for i in range(6):
  plt.subplot(2, 3, i+1)
  plt.imshow(images[i])
  plt.title(titles[i])
  plt.axis("off")

plt.show()

* #### 高通濾波器 High Pass Filter
 - 保持高頻部分，減少或消除低頻部分

* #### Sobel Filter -- Edge Detection

In [ ]:
# upload bicycle.jpg
!gdown --id 1nyJIv5Wu4jHwxVvsUqeSzYee_QDhmPTV

In [ ]:
img = cv2.imread("bicycle.jpg")[:, :, ::-1]

In [ ]:
sobel_filter_h = np.array([[1, 2, 1],
                [0, 0, 0],
                [-1, -2, -1]])
sobel_filter_v = np.array([[1, 0, -1],
                [2, 0, -2],
                [1, 0, -1]])

laplacian_filter = np.array([[-1, -1, -1],
                [-1, 8, -1],
                [-1, -1, -1]])

In [ ]:
edge_h = cv2.filter2D(img, -1, sobel_filter_h)
edge_v = cv2.filter2D(img, -1, sobel_filter_v)
edge_l = cv2.filter2D(img, -1, laplacian_filter)

In [ ]:
output = [img, edge_h, edge_v, edge_l]
titles = ['Original', 'Horizantol', 'Vertical', 'Laplacian']

plt.figure(figsize=(24, 8))
for i in range(4):
  plt.subplot(1, 4, i+1)
  plt.imshow(output[i], cmap='gray')
  plt.title(titles[i], fontsize=16)
  plt.axis("off")
plt.show()

* #### Laplacian Filter

In [ ]:
# upload poker.jpg
!gdown --id 1HQx3X5AmckcFCrtunKYcUGzm2L5OC53g

In [ ]:
img = cv2.imread("poker.jpg")[:, :, ::-1]

# Laplacian Filter
laplacian_filter = -np.ones((3, 3))
laplacian_filter[1, 1] = 8
print(laplacian_filter)

In [ ]:
cv2.filter2D

In [ ]:
edge = cv2.filter2D(img, -1, laplacian_filter)

In [ ]:
output = [img, edge]
titles = ['Original', 'Edges']
plt.figure(figsize=(16, 8))
for i in range(2):
  plt.subplot(1, 2, i+1)
  plt.imshow(output[i], cmap='gray')
  plt.title(titles[i], fontsize=16)
  plt.axis("off")
plt.show()

* #### Laplacian --Sharpening 影像銳化

In [ ]:
img = cv2.imread("Einstein.jpg")[:, :, ::-1]
edge = cv2.Laplacian(img, cv2.CV_64F)
sharp = img - 0.5 * edge

In [ ]:
sharp[sharp > 255] = 255
sharp[sharp < 0] = 0

In [ ]:
output = [img, sharp.astype('uint8')]
titles = ['Original', 'Sharpening']
plt.figure(figsize=(16, 8))
for i in range(2):
  plt.subplot(1, 2, i+1)
  plt.imshow(output[i], cmap='gray')
  plt.title(titles[i], fontsize=16)
  plt.axis("off")
plt.show()

Reference<br>
[1] http://163.13.127.10/pages/cht/courses/yen/proj105/pdf/day3.pdf